In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/temp-data/wavelet_features_reduced.xlsx


In [2]:
!pip install tqdm psutil


In [3]:
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import psutil
import time

# Function to monitor memory usage
def memory_usage():
    process = psutil.Process()
    memory = process.memory_info().rss / 1024 / 1024  # in MB
    return f"Memory usage: {memory:.2f} MB"

# Load dataset
df = pd.read_excel("/kaggle/input/temp-data/wavelet_features_reduced.xlsx")

# Rename columns (adjust if necessary)
df.columns = ["Person", "Image", "Mean", "Variance", "Energy"]

# Features and label
X = df[["Mean", "Variance", "Energy"]]
le = LabelEncoder()
y = le.fit_transform(df["Person"])  # Encode person IDs

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("dataset loading done")

dataset loading done


In [4]:
X_train

,Mean,Variance,Energy
401907,0.160383,0.034157,1119.724041
306750,0.162920,0.017914,1018.263479
569722,0.206594,0.023451,1593.000285
571340,0.165008,0.048454,1284.971977
226391,0.219424,0.046007,1957.080068
...,...,...,...
259178,0.212076,0.053807,1916.190724
365838,0.228050,0.039063,2021.086069
131932,0.217715,0.022241,1735.072464
671155,0.172725,0.023107,1169.313511


In [5]:
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tqdm import tqdm
import psutil
import time
import numpy as np

# Function to monitor memory usage
def memory_usage():
    process = psutil.Process()
    memory = process.memory_info().rss / 1024 / 1024  # in MB
    return memory

# Start timing
start_time = time.time()

# Load dataset (commented out since assumed loaded)
# df = pd.read_excel("/kaggle/input/temp-data/wavelet_features_reduced.xlsx")
print("Dataset loading assumed complete.")

# Features and label (uncomment for actual data)
# X = df[["Mean", "Variance", "Energy"]]
# le = LabelEncoder()
# y = le.fit_transform(df["Person"])

# Split data (uncomment for actual data)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Simulate data for runtime estimation (replace with your actual X_train, y_train)
n_samples, n_features = 560000, 3
X_train = np.random.rand(n_samples, n_features)  # Replace with actual data
y_train = np.random.randint(0, 10, n_samples)    # Replace with actual labels
X_test = np.random.rand(int(n_samples * 0.25), n_features)
y_test = np.random.randint(0, 10, int(n_samples * 0.25))

# Scale features (critical for SGDClassifier)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(f"Scaling done. {memory_usage():.2f} MB")

# Initialize SGDClassifier with tuned parameters
clf = SGDClassifier(
    loss='log_loss',
    max_iter=10000,  # More iterations for better convergence
    tol=1e-3,
    learning_rate='adaptive',  # Adjusts learning rate dynamically
    eta0=0.01,  # Initial learning rate
    alpha=0.0001,  # Regularization
    n_jobs=-1,
    random_state=42
)

# Train with partial_fit for memory efficiency
batch_size = 10000
n_batches = len(X_train) // batch_size + 1

print("Training started...")
for i in tqdm(range(n_batches)):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(X_train))
    if start_idx < len(X_train):
        clf.partial_fit(X_train[start_idx:end_idx], y_train[start_idx:end_idx], classes=np.unique(y_train))
print(f"Training done. {memory_usage():.2f} MB")

# Predict and evaluate
y_pred = clf.predict(X_test)
print(f"Prediction done. {memory_usage():.2f} MB")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, zero_division=0))

# End timing
end_time = time.time()
runtime = end_time - start_time
print(f"\nTotal runtime: {runtime:.2f} seconds")
print(f"Final memory usage: {memory_usage():.2f} MB")

Dataset loading assumed complete.
Scaling done. 682.09 MB
Training started...


100%|██████████| 57/57 [00:01<00:00, 38.18it/s]


Training done. 684.32 MB
Prediction done. 695.95 MB

Classification Report:
              precision    recall  f1-score   support

           0       0.10      0.20      0.13     14003
           1       0.11      0.01      0.02     13875
           2       0.10      0.15      0.12     13814
           3       0.10      0.21      0.14     14107
           4       0.00      0.00      0.00     14034
           5       0.00      0.00      0.00     14127
           6       0.10      0.34      0.16     14011
           7       0.10      0.04      0.06     13836
           8       0.00      0.00      0.00     14058
           9       0.10      0.05      0.07     14135

    accuracy                           0.10    140000
   macro avg       0.07      0.10      0.07    140000
weighted avg       0.07      0.10      0.07    140000


Total runtime: 1.87 seconds
Final memory usage: 695.95 MB


In [6]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import psutil
import time
import numpy as np
import tracemalloc

# Function to monitor memory usage
def memory_usage():
    process = psutil.Process()
    memory = process.memory_info().rss / 1024 / 1024  # in MB
    return memory

# Start memory tracking
tracemalloc.start()

# Start timing
start_time = time.time()

# Load dataset (uncomment for actual data)
# df = pd.read_excel("/kaggle/input/temp-data/wavelet_features_reduced.xlsx")
print("Dataset loading assumed complete.")

# Features and label (uncomment for actual data)
# X = df[["Mean", "Variance", "Energy"]]
# le = LabelEncoder()
# y = le.fit_transform(df["Person"])

# Split data (uncomment for actual data)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Simulate data for runtime estimation (replace with your actual X_train, y_train)
n_samples, n_features = 560000, 3
X_train = np.random.rand(n_samples, n_features)  # Replace with actual data
y_train = np.random.randint(0, 10, n_samples)    # Replace with actual labels
X_test = np.random.rand(int(n_samples * 0.25), n_features)
y_test = np.random.randint(0, 10, int(n_samples * 0.25))

# Note: RandomForest doesn't require scaling, so we skip StandardScaler

# Initialize RandomForestClassifier
clf = RandomForestClassifier(
    n_estimators=150,  # More trees for better accuracy and longer runtime
    max_depth=15,      # Deeper trees for better performance
    min_samples_split=10,
    n_jobs=-1,         # Use all CPU cores
    random_state=42
)

# Train
print("Training started...")
clf.fit(X_train, y_train)
print(f"Training done. {memory_usage():.2f} MB")

# Predict and evaluate
y_pred = clf.predict(X_test)
print(f"Prediction done. {memory_usage():.2f} MB")

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, zero_division=0))

# End timing
end_time = time.time()
runtime = end_time - start_time
print(f"\nTotal runtime: {runtime:.2f} seconds")

# Memory stats
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage: {current / 1024 / 1024:.2f} MB")
print(f"Peak memory usage: {peak / 1024 / 1024:.2f} MB")
tracemalloc.stop()

Dataset loading assumed complete.
Training started...
Training done. 914.56 MB
Prediction done. 926.41 MB

Classification Report:
              precision    recall  f1-score   support

           0       0.10      0.07      0.08     14001
           1       0.10      0.06      0.08     13892
           2       0.10      0.13      0.11     13979
           3       0.10      0.07      0.09     14002
           4       0.10      0.10      0.10     13938
           5       0.10      0.19      0.13     14097
           6       0.10      0.19      0.13     14265
           7       0.10      0.06      0.07     14052
           8       0.10      0.06      0.08     13755
           9       0.10      0.08      0.09     14019

    accuracy                           0.10    140000
   macro avg       0.10      0.10      0.10    140000
weighted avg       0.10      0.10      0.10    140000


Total runtime: 74.92 seconds
Current memory usage: 22.66 MB
Peak memory usage: 205.22 MB
